# Mixed beverage data for June 2017


- Month studied: June 2017
- File used: July 2017, MIXEDBEV_07_2017.CSV

The data was downloaded from [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) on the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This is a stripped down version (compared to the original fork) that skips explanation of steps beyond commenting.

## Get to the goods

To skip most of the setup code and get to what you really want, search for:

- Top sales statewide
- Austin sales
- Central Texas cities

### File download

- Go to the [Texas Comptroller data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/) and copy the url for the CSV for this month and enter it below.
- You also need to set to set the [processing variables](Processing-variables) for this month.

In [1]:
## %%bash
## downloads the mixedbev file into mixbev-files folder
## You have to set this URL based on location in data center

## these are commented out now. I've been pulling them manually
## and processing from those file
# cd ../test/
# curl -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV

In [2]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Processing variables
Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

Here are the files we have downloaded:

In [3]:
ls ../mixbev-files/2017/

MIXEDBEV_01_2017.CSV  MIXEDBEV_04_2015.CSV  MIXEDBEV_06_2017.CSV
MIXEDBEV_02_2017.CSV  MIXEDBEV_04_2017.CSV  MIXEDBEV_07_2017.CSV
MIXEDBEV_03_2017.CSV  MIXEDBEV_05_2017.CSV


In [4]:
# this is our source file, which may have been downloaded above
# Swap out the file name here and date below as needed
file = '../mixbev-files/2017/MIXEDBEV_07_2017.CSV'
file_name = ''
online_name = ''

# setting the month_studied var.
# This should be checked in the table below that counts records by month
month_studied = '2017/06'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

### import and processing

In [5]:
# sets the column names of the original data set.
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
# Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

# mixbev_trim creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them of white space,
# adding them to the end of the table with new names.
# The last computation does the math to create the Gross Receipts based on the tax_rate set above

mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip())),
    ('Receipts_compute', agate.Formula(agate.Number(), lambda r: (r['Report Tax'] / tax_rate) * 100))
])

# the Receipts_compute computation above returns as a decimal number,
# so this function rounds those numbers.
# I might refactor this later so I can use it elsewhere.
def round_receipt(row):
    return row['Receipts_compute'].quantize(Decimal('0.01'))

# This compute method uses round_recipt function above,
# putting the results into a new table.
mixbev_round = mixbev_trim.compute([
    ('Receipts', agate.Formula(agate.Number(), round_receipt))
])

# creates new table, selecting just the columns we need
# then renames some of them for ease later.
mixbev_cleaned = mixbev_round.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Report Period',
    'Report Tax',
    'Receipts'
]).rename(column_names = {
    'Location State': 'State',
    'Location Zip Code': 'Zip',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

# Concatenates the name and address
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(Name)s %(Address)s' % row))
])

# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('../resource-files/counties.csv', column_types={'code': agate.Text()})

# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'Zip',
    'county',
    'Period',
    'Tax',
    'Receipts'
]).rename(column_names = {
    'county': 'County'
})


### Looking at dates of the records

This basically confirms that the file has multiple dates, and that we are looking at the right month of data. Typically a data set will have mostly reports from the previous month, but there are always also submissions from other months. We want to filter out those other months, which we do based on the `month_studied` variable set near the top of the file, which should match the period at the top of the table below.


In [6]:
# Pivot the mixbev table by Period. Default it give a Count of the records
# We then order the table by Count in descending order
by_period = mixbev.pivot('Period').order_by('Count', reverse=True)

# prints the table of period and number of records
by_period.limit(5).print_table(max_rows=None)

| Period  |  Count |
| ------- | ------ |
| 2017/06 | 14,380 |
| 2017/05 |  1,473 |
| 2017/04 |    224 |
| 2017/03 |     91 |
| 2017/02 |     52 |


In [7]:
## filters the records to our month_studied
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)

# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev_month.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Tax_sum', agate.Sum('Tax')),
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to computer the Tax and Receipts columns.

In [8]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev_month.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Tax_sum', agate.Sum('Tax')),
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# summing sales statewide for month
print('\nTotal sales across the state for the given month: {}\n'.format(
    mixbev_month.aggregate(agate.Sum('Receipts'))
))

print('Top sales by establishment statewide\n')

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=35)


Total sales across the state for the given month: 542608571.83

Top sales by establishment statewide

| Establishment                       | County     | City            |    Tax_sum |    Sales_sum |
| ----------------------------------- | ---------- | --------------- | ---------- | ------------ |
| AT&T STADIUM 1 LEGENDS WAY          | Tarrant    | ARLINGTON       | 130,637.20 | 1,949,808.96 |
| GAYLORD TEXAN 1501 GAYLORD TRL      | Tarrant    | GRAPEVINE       | 125,892.06 | 1,878,985.97 |
| HOSPITALITY INTERNATIONAL, INC 2... | Bexar      | SAN ANTONIO     |  98,594.18 | 1,471,554.93 |
| ARAMARK SPORTS & ENTERTAINMENT 5... | Harris     | HOUSTON         |  89,875.20 | 1,341,420.90 |
| SPECTRUM CATERING 27328 SPECTRUM... | Montgomery | OAK RIDGE NORTH |  71,130.28 | 1,061,645.97 |
| SALC, INC. 2201 N STEMMONS FWY FL 1 | Dallas     | DALLAS          |  69,297.49 | 1,034,290.90 |
| METROPLEX SPORTSERVICE, INC. 100... | Tarrant    | ARLINGTON       |  63,345.95 |   945,461.94 |
| WLS 

## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [9]:
# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')

print('\nTotal sales across the state for the given month: {}\n'.format(
    austin.aggregate(agate.Sum('Receipts_sum'))
))

# print the resulting table
austin.limit(10).print_table(max_column_width=50)


Total sales across the state for the given month: 59471360.42

| Establishment                                      | City   |   Tax_sum | Receipts_sum |
| -------------------------------------------------- | ------ | --------- | ------------ |
| WLS BEVERAGE CO 110 E 2ND ST                       | AUSTIN | 61,596.58 |   919,351.94 |
| RYAN SANDERS SPORTS SERVICES, 9201 CIRCUIT OF T... | AUSTIN | 40,867.25 |   609,958.96 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | AUSTIN | 36,399.15 |   543,270.90 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | AUSTIN | 30,668.71 |   457,741.94 |
| TOP GOLF 2700 ESPERANZA XING                       | AUSTIN | 29,364.22 |   438,271.94 |
| W HOTEL AUSTIN 200 LAVACA ST                       | AUSTIN | 28,373.42 |   423,483.88 |
| OASIS CANTINA DEL LAGO 6550 COMANCHE TRL           | AUSTIN | 27,600.24 |   411,943.88 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | AUSTIN | 25,350.58 |   378,366.87 |
| THE BLIND PIG PUB 317 E 

## More Central Texas cities

In [10]:
location_sum('City', 'BASTROP').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PIN 931 MA... | BASTROP | 4,064.01 |    60,656.87 |
| CHILI'S GRILL & BAR 734 HIGHWAY 71 W     | BASTROP | 2,623.25 |    39,152.99 |
| NEIGHBOR'S 601 CHESTNUT ST UNIT C        | BASTROP | 2,322.15 |    34,658.96 |


In [11]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=40)

| Establishment                            | City     |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| CAFE BLUE 12800 HILL COUNTRY BLVD STE    | BEE CAVE | 5,583.71 |    83,338.96 |
| MAUDIE'S HILL COUNTRY, LLC 12506 SHOP... | BEE CAVE | 5,456.68 |    81,442.99 |
| HCG BEVERAGE, LLC 12525 BEE CAVE PKWY    | BEE CAVE | 4,882.75 |    72,876.87 |


In [12]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=40)

| Establishment                            | City |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---- | -------- | ------------ |
| BUCKS BACKYARD 1750 S FM 1626            | BUDA | 7,420.04 |   110,746.87 |
| WILLIE'S JOINT 824 MAIN ST               | BUDA | 4,255.50 |    63,514.93 |
| NATE'S AT BUDA MILL AND GRAIN, 306 S ... | BUDA | 3,504.23 |    52,301.94 |


In [13]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |   Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | --------- | ------------ |
| CHUY'S 4911 183A TOLL RD                 | CEDAR PARK | 10,565.29 |   157,690.90 |
| LUPE TORTILLA MEXICAN RESTAURA 4501 1... | CEDAR PARK |  7,346.28 |   109,645.97 |
| BLUE CORN HARVEST BAR & GRILL 700 E W... | CEDAR PARK |  5,188.21 |    77,435.97 |


In [14]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=40)

| Establishment                            | City             |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------------- | -------- | ------------ |
| DEEP EDDY DISTILLING CO 2250 E HIGHWA... | DRIPPING SPRINGS | 4,530.87 |    67,624.93 |
| TRUDY'S FOUR STAR 13059 FOUR STAR BLVD   | DRIPPING SPRINGS | 4,173.09 |    62,284.93 |
| TREATY OAK BREWING AND DISTILL 16604 ... | DRIPPING SPRINGS | 4,094.90 |    61,117.91 |


In [15]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=40)

| Establishment                      | City       |  Tax_sum | Receipts_sum |
| ---------------------------------- | ---------- | -------- | ------------ |
| EL MONUMENTO 205 W 2ND ST          | GEORGETOWN | 6,575.78 |    98,145.97 |
| HARDTAILS 1515 N IH 35             | GEORGETOWN | 4,919.87 |    73,430.90 |
| DOS SALSAS CAFE INC 1104 S MAIN ST | GEORGETOWN | 4,862.05 |    72,567.91 |


In [16]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=40)

| Establishment                            | City |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---- | -------- | ------------ |
| CASA GARCIA'S MEXICAN RESTAURA 5401 F... | KYLE | 5,344.45 |    79,767.91 |
| EVO ENTERTAINMENT CENTER 3200 KYLE XING  | KYLE | 4,290.21 |    64,032.99 |
| APPLEJAM BEVERAGE 5363 KYLE CENTER DR    | KYLE | 1,534.63 |    22,904.93 |


In [17]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| COPPERHEAD GRILL 6115 LOHMANS FORD RD    | LAGO VISTA | 1,082.58 |    16,157.91 |
| MARIA'S BAR & GRILL MEXICAN RE 20602 ... | LAGO VISTA |   483.74 |     7,220.00 |


In [18]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |   Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | --------- | ------------ |
| LAKEWAY RESORT AND SPA 101 LAKEWAY DR    | LAKEWAY | 11,646.34 |   173,825.97 |
| THE GROVE WINE BAR AND KITCHEN 3001 R... | LAKEWAY |  7,532.74 |   112,428.96 |
| CANYON GRILLE AT ROUGH HOLLOW, 103 YA... | LAKEWAY |  4,414.49 |    65,887.91 |


In [19]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA 3550 LAKELI... | LEANDER | 3,763.12 |    56,165.97 |
| JARDIN DEL REY 703 S HIGHWAY 183         | LEANDER | 2,854.46 |    42,603.88 |
| TAPATIA JALISCO #3 LLC 651 N US 183      | LEANDER | 1,696.37 |    25,318.96 |


In [20]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=40)

| Establishment                            | City         |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------ | -------- | ------------ |
| JARDIN CORONA 15395 W STATE HIGHWAY 29   | LIBERTY HILL | 2,869.07 |    42,821.94 |
| MARGARITA'S RESTAURANT 10280 W STATE ... | LIBERTY HILL | 2,468.34 |    36,840.90 |
| ELENAS MEXICAN RESTAURANT 14801 W STA... | LIBERTY HILL |   132.79 |     1,981.94 |


In [21]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=40)

| Establishment                            | City         |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------ | -------- | ------------ |
| MAVERICKS 1700 GRAND AVENUE PKWY STE 2   | PFLUGERVILLE | 8,164.62 |   121,860.00 |
| HANOVER'S DRAUGHT HAUS 108 E MAIN ST     | PFLUGERVILLE | 5,133.54 |    76,620.00 |
| RIO GRANDE TEX MEX 15821 CENTRAL COMM... | PFLUGERVILLE | 4,149.10 |    61,926.87 |


In [22]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |   Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | --------- | ------------ |
| CHUY'S ROUND ROCK 2320 N INTERSTATE 35   | ROUND ROCK | 11,632.87 |   173,624.93 |
| RICK'S CABARET 3105 S INTERSTATE 35      | ROUND ROCK | 10,318.53 |   154,007.91 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA... | ROUND ROCK | 10,106.54 |   150,843.88 |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL      | ROUND ROCK |  9,212.83 |   137,504.93 |
| THIRD BASE ROUND ROCK, LLC 3107 S INT... | ROUND ROCK |  9,011.96 |   134,506.87 |


In [23]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| THE MARC 120 E SAN ANTONIO ST            | SAN MARCOS | 8,312.02 |   124,060.00 |
| ZELICKS 336 W HOPKINS ST                 | SAN MARCOS | 7,891.12 |   117,777.91 |
| PLUCKERS WING BAR 105 N INTERSTATE 35    | SAN MARCOS | 7,028.90 |   104,908.96 |
| CHUY'S SAN MARCOS 1121 N INTERSTATE 35   | SAN MARCOS | 6,849.07 |   102,224.93 |
| 54TH STREET RESTAURANT & DRAFT 1303 S... | SAN MARCOS | 6,635.54 |    99,037.91 |


In [24]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=40)

| Establishment                            | City      |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------- | -------- | ------------ |
| ANGEL'S ICEHOUSE 21815 W HWY 71          | SPICEWOOD | 3,980.47 |    59,410.00 |
| POODIES HILLTOP ROADHOUSE 22308 STATE... | SPICEWOOD | 3,096.00 |    46,208.96 |
| APIS RESTAURANT 23526 STATE HIGHWAY 71 W | SPICEWOOD | 2,022.26 |    30,182.99 |


In [25]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=40)

| Establishment                            | City          |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------- | -------- | ------------ |
| DOC'S BACKYARD 5207 BRODIE LN # 100      | SUNSET VALLEY | 5,089.85 |    75,967.91 |
| BJ'S RESTAURANT AND BREWHOUSE 5207 BR... | SUNSET VALLEY | 4,199.89 |    62,684.93 |
| CANE ROSSO 4715 S LAMAR BLVD STE 103     | SUNSET VALLEY | 1,899.31 |    28,347.91 |


In [26]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=40)

| Establishment                            | City            |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------------- | -------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURA 701 S ... | WEST LAKE HILLS | 6,027.52 |    89,962.99 |
| CHIPOTLE CHIPOTLE MEXICAN GRIL 3300 B... | WEST LAKE HILLS |    26.39 |       393.88 |


## Sales by county example

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [27]:
location_sum('County', 'CALDWELL').limit(3).print_table(max_column_width=40)

| Establishment                            | County   |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| GUADALAJARA MEXICAN RESTAURANT 1710 S... | Caldwell | 1,311.59 |    19,575.97 |
| THE PEARL 110 N MAIN ST                  | Caldwell |   709.39 |    10,587.91 |
| MR TACO 1132 E PIERCE ST                 | Caldwell |   610.83 |     9,116.87 |


## Sales by ZIP Code
Just making sure that 78701 is at the top of this list, which it has been every month for a decade.

In [28]:
# top zip code gross receipts
zip_receipts = mixbev_month.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()

| Zip   |           Sum |
| ----- | ------------- |
| 78701 | 20,911,901.08 |
| 75201 | 10,600,247.40 |
| 78205 |  9,943,626.36 |
| 77002 |  9,390,179.42 |
| 77006 |  7,158,393.46 |
